<a href="https://colab.research.google.com/github/THashiz/m5-ia/blob/master/M5_cal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M5 カレンダー分析

In [0]:
# kaggle API key upload
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [0]:
!rm -rf ~/.kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!pip install kaggle

In [0]:
# Competitionのデータをダウンロード
!kaggle competitions download -c m5-forecasting-accuracy

In [0]:
!unzip /content/sample_submission.csv.zip
!unzip /content/sell_prices.csv.zip
!unzip /content/sales_train_validation.csv.zip

In [0]:
!apt-get update
!apt-get install python-pydot -y --allow-unauthenticated

In [0]:
# ライブラリのロード
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
from IPython.display import display, Image, display_png
import io

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# ローカルのファイルをロード
pd.set_option('display.max_columns', 200)
#sales_train = pd.read_csv('/content/sales_train_validation.csv')
calender = pd.read_csv('/content/calendar.csv')
#submission = pd.read_csv('/content/sample_submission.csv')
#prices = pd.read_csv('/content/sell_prices.csv')
#evaluation = pd.read_csv('/content/sales_train_evaluation.csv')


## calenderの分析

In [0]:
calender.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [0]:
# event1, event2を集約する
event1 = calender.loc[ : , ['year','date','event_type_1','event_name_1']].rename(columns={'event_type_1': 'event_type','event_name_1': 'event_name'}).dropna()
event2 = calender.loc[ : , ['year','date','event_type_2','event_name_2']].rename(columns={'event_type_2': 'event_type','event_name_2': 'event_name'}).dropna()
events = event1.append(event2)

In [0]:
uniqEvents=events.groupby(['event_name','year','date']).count()
display(uniqEvents)

event_type
event_name   year date                  
Chanukah End 2011 2011-12-28           1
             2012 2012-12-16           1
             2013 2013-12-05           1
             2014 2014-12-24           1
             2015 2015-12-14           1
...                                  ...
VeteransDay  2011 2011-11-11           1
             2012 2012-11-11           1
             2013 2013-11-11           1
             2014 2014-11-11           1
             2015 2015-11-11           1

[167 rows x 1 columns]

In [0]:
uniqEvents.to_csv('/content/holiday.csv')

In [0]:
# 手ゴネで作成したevents.csvをロード
#url = 'https://github.com/THashiz/m5-ia/blob/master/events.csv'
uploaded = files.upload()

In [0]:
events = pd.read_csv('/content/events.csv')

In [0]:
events.head()

,org_event_name,year,date,event_name,from,to
0,Chanukah End,2011,2011-12-28,Chanukah,2011-12-20,2011-12-28
1,Chanukah End,2012,2012-12-16,Chanukah,2012-12-08,2012-12-16
2,Chanukah End,2013,2013-12-05,Chanukah,2013-11-27,2013-12-05
3,Chanukah End,2014,2014-12-24,Chanukah,2014-12-16,2014-12-24
4,Chanukah End,2015,2015-12-14,Chanukah,2015-12-06,2015-12-14


In [0]:
event_type1 = calender.loc[ : , ['event_type_1','event_name_1']].rename(columns={'event_type_1': 'event_type','event_name_1': 'org_event_name'}).dropna()
event_type2 = calender.loc[ : , ['event_type_2','event_name_2']].rename(columns={'event_type_2': 'event_type','event_name_2': 'org_event_name'}).dropna()
event_types= event_type1.append(event_type2).drop_duplicates()

add_type = pd.merge(events, event_types, on='org_event_name')

In [0]:
new_events = add_type.loc[ : , ['event_type','event_name','year','from','to']].rename(columns={'from': 'from_day','to': 'to_day'}).dropna()

In [0]:
display(new_events)

,event_type,event_name,year,from_day,to_day
0,Religious,Chanukah,2011,2011-12-20,2011-12-28
1,Religious,Chanukah,2012,2012-12-08,2012-12-16
2,Religious,Chanukah,2013,2013-11-27,2013-12-05
3,Religious,Chanukah,2014,2014-12-16,2014-12-24
4,Religious,Chanukah,2015,2015-12-06,2015-12-14
...,...,...,...,...,...
162,National,VeteransDay,2011,2011-11-11,2011-11-11
163,National,VeteransDay,2012,2012-11-11,2012-11-11
164,National,VeteransDay,2013,2013-11-11,2013-11-11
165,National,VeteransDay,2014,2014-11-11,2014-11-11


In [0]:
tmp_calendar = calender.loc[ : , ['date']].dropna()

In [0]:
display(tmp_calendar)

,date
0,2011-01-29
1,2011-01-30
2,2011-01-31
3,2011-02-01
4,2011-02-02
...,...
1964,2016-06-15
1965,2016-06-16
1966,2016-06-17
1967,2016-06-18


In [0]:
import sqlite3
drive_path = '/content/cal.db'
conn = sqlite3.connect(drive_path)

In [0]:
tmp_calendar.to_sql('tmp_calendar', conn, schema=None, if_exists='replace', index=False)

In [0]:
new_events.to_sql('new_events', conn, schema=None, if_exists='replace', index=False)

In [0]:
# event_type毎にイベント日にフラグを立てる。イベントが重複しても1となる。
qry = '''
    select 
        date,
        case when Cultural > 0  then 1 else 0 end as Cultural,
        case when National > 0  then 1 else 0 end as National,
        case when Religious > 0 then 1 else 0 end as Religious,
        case when Sporting > 0  then 1 else 0 end as Sporting
    from (
    select
        date,
        sum(Cultural) Cultural,
        sum(National) National,
        sum(Religious) Religious,
        sum(Sporting) Sporting
    from (
    select  
        tmp_calendar.date,
        case when new_events.event_type = 'Cultural'  then 1 else 0 end as Cultural,
        case when new_events.event_type = 'National'  then 1 else 0 end as National,
        case when new_events.event_type = 'Religious' then 1 else 0 end as Religious,
        case when new_events.event_type = 'Sporting'  then 1 else 0 end as Sporting
    from tmp_calendar  
    left outer join new_events
     on 
        tmp_calendar.date between new_events.from_day and new_events.to_day
    )
    group by date
    order by date
    )
    '''
event_type_calendar = pd.read_sql_query(qry, conn)

In [0]:
event_type_calendar.count()

date         1969
Cultural     1969
National     1969
Religious    1969
Sporting     1969
dtype: int64

In [0]:
event_type_calendar.head()

,date,Cultural,National,Religious,Sporting
0,2011-01-29,0,0,0,0
1,2011-01-30,0,0,0,0
2,2011-01-31,0,0,0,0
3,2011-02-01,0,0,0,0
4,2011-02-02,0,0,0,0


In [0]:
# 保存
event_type_calendar.to_csv('/content/event_type_calendar.csv')

In [0]:
# event_type毎にイベント日にフラグを立てる。イベントが重複しても1となる。
qry = '''
    select
        date,
        sum(Chanukah) as Chanukah,
        sum(Christmas) as Christmas,
        sum(CincoDeMayo) as CincoDeMayo,
        sum(ColumbusDay) as ColumbusDay,
        sum(Easter) as Easter,
        sum(EidAlFitr) as EidAlFitr,
        sum(EidAlAdha) as EidAlAdha,
        sum(FathersDay) as FathersDay,
        sum(Halloween) as Halloween,
        sum(IndependenceDay) as IndependenceDay,
        sum(LaborDay) as LaborDay,
        sum(Lent) as Lent,
        sum(MartinLutherKingDay) as MartinLutherKingDay,
        sum(MemorialDay) as MemorialDay,
        sum(MothersDay) as MothersDay,
        sum(NBAFinal) as NBAFinal,
        sum(NewYear) as NewYear,
        sum(OrthodoxChristmas) as OrthodoxChristmas,
        sum(OrthodoxEaster) as OrthodoxEaster,
        sum(Pesach) as Pesach,
        sum(PresidentsDay) as PresidentsDay,
        sum(Purim) as Purim,
        sum(Ramadan) as Ramadan,
        sum(StPatricksDay) as StPatricksDay,
        sum(SuperBowl) as SuperBowl,
        sum(Thanksgiving) as Thanksgiving,
        sum(ValentinesDay) as ValentinesDay,
        sum(VeteransDay) as VeteransDay
    from (
    select  
        tmp_calendar.date,
        case when new_events.event_name = 'Chanukah' then 1 else 0 end as Chanukah,
        case when new_events.event_name = 'Christmas' then 1 else 0 end as Christmas,
        case when new_events.event_name = 'CincoDeMayo' then 1 else 0 end as CincoDeMayo,
        case when new_events.event_name = 'ColumbusDay' then 1 else 0 end as ColumbusDay,
        case when new_events.event_name = 'Easter' then 1 else 0 end as Easter,
        case when new_events.event_name = 'EidAlFitr' then 1 else 0 end as EidAlFitr,
        case when new_events.event_name = 'EidAlAdha' then 1 else 0 end as EidAlAdha,
        case when new_events.event_name = 'FathersDay' then 1 else 0 end as FathersDay,
        case when new_events.event_name = 'Halloween' then 1 else 0 end as Halloween,
        case when new_events.event_name = 'IndependenceDay' then 1 else 0 end as IndependenceDay,
        case when new_events.event_name = 'LaborDay' then 1 else 0 end as LaborDay,
        case when new_events.event_name = 'Lent' then 1 else 0 end as Lent,
        case when new_events.event_name = 'MartinLutherKingDay' then 1 else 0 end as MartinLutherKingDay,
        case when new_events.event_name = 'MemorialDay' then 1 else 0 end as MemorialDay,
        case when new_events.event_name = 'MothersDay' then 1 else 0 end as MothersDay,
        case when new_events.event_name = 'NBAFinal' then 1 else 0 end as NBAFinal,
        case when new_events.event_name = 'NewYear' then 1 else 0 end as NewYear,
        case when new_events.event_name = 'OrthodoxChristmas' then 1 else 0 end as OrthodoxChristmas,
        case when new_events.event_name = 'OrthodoxEaster' then 1 else 0 end as OrthodoxEaster,
        case when new_events.event_name = 'Pesach' then 1 else 0 end as Pesach,
        case when new_events.event_name = 'PresidentsDay' then 1 else 0 end as PresidentsDay,
        case when new_events.event_name = 'Purim' then 1 else 0 end as Purim,
        case when new_events.event_name = 'Ramadan' then 1 else 0 end as Ramadan,
        case when new_events.event_name = 'StPatricksDay' then 1 else 0 end as StPatricksDay,
        case when new_events.event_name = 'SuperBowl' then 1 else 0 end as SuperBowl,
        case when new_events.event_name = 'Thanksgiving' then 1 else 0 end as Thanksgiving,
        case when new_events.event_name = 'ValentinesDay' then 1 else 0 end as ValentinesDay,
        case when new_events.event_name = 'VeteransDay' then 1 else 0 end as VeteransDay
    from tmp_calendar  
    left outer join new_events
     on 
        tmp_calendar.date between new_events.from_day and new_events.to_day
    )
    group by date
    order by date
    '''
event_name_calendar = pd.read_sql_query(qry, conn)

In [0]:
event_name_calendar.count()

date                   1969
Chanukah               1969
Christmas              1969
CincoDeMayo            1969
ColumbusDay            1969
Easter                 1969
EidAlFitr              1969
EidAlAdha              1969
FathersDay             1969
Halloween              1969
IndependenceDay        1969
LaborDay               1969
Lent                   1969
MartinLutherKingDay    1969
MemorialDay            1969
MothersDay             1969
NBAFinal               1969
NewYear                1969
OrthodoxChristmas      1969
OrthodoxEaster         1969
Pesach                 1969
PresidentsDay          1969
Purim                  1969
Ramadan                1969
StPatricksDay          1969
SuperBowl              1969
Thanksgiving           1969
ValentinesDay          1969
VeteransDay            1969
dtype: int64

In [0]:
event_name_calendar.head()

,date,Chanukah,Christmas,CincoDeMayo,ColumbusDay,Easter,EidAlFitr,EidAlAdha,FathersDay,Halloween,IndependenceDay,LaborDay,Lent,MartinLutherKingDay,MemorialDay,MothersDay,NBAFinal,NewYear,OrthodoxChristmas,OrthodoxEaster,Pesach,PresidentsDay,Purim,Ramadan,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay
0,2011-01-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2011-01-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2011-01-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2011-02-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2011-02-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
event_name_calendar.to_csv('/content/event_name_calendar.csv')

In [0]:
# event_type毎にイベント日にフラグを立てる。イベントが重複しても1となる。
qry = '''
    select
        date,
        sum(Chanukah) as E01,
        sum(Christmas) as E02,
        sum(CincoDeMayo) as E03,
        sum(ColumbusDay) as E04,
        sum(Easter) as E05,
        sum(EidAlFitr) as E06,
        sum(EidAlAdha) as E07,
        sum(FathersDay) as E08,
        sum(Halloween) as E09,
        sum(IndependenceDay) as E10,
        sum(LaborDay) as E11,
        sum(Lent) as E12,
        sum(MartinLutherKingDay) as E13,
        sum(MemorialDay) as E14,
        sum(MothersDay) as E15,
        sum(NBAFinal) as E16,
        sum(NewYear) as E17,
        sum(OrthodoxChristmas) as E18,
        sum(OrthodoxEaster) as E19,
        sum(Pesach) as E20,
        sum(PresidentsDay) as E21,
        sum(Purim) as E22,
        sum(Ramadan) as E23,
        sum(StPatricksDay) as E24,
        sum(SuperBowl) as E25,
        sum(Thanksgiving) as E26,
        sum(ValentinesDay) as E27,
        sum(VeteransDay) as E28
    from (
    select  
        tmp_calendar.date,
        case when new_events.event_name = 'Chanukah' then 1 else 0 end as Chanukah,
        case when new_events.event_name = 'Christmas' then 1 else 0 end as Christmas,
        case when new_events.event_name = 'CincoDeMayo' then 1 else 0 end as CincoDeMayo,
        case when new_events.event_name = 'ColumbusDay' then 1 else 0 end as ColumbusDay,
        case when new_events.event_name = 'Easter' then 1 else 0 end as Easter,
        case when new_events.event_name = 'EidAlFitr' then 1 else 0 end as EidAlFitr,
        case when new_events.event_name = 'EidAlAdha' then 1 else 0 end as EidAlAdha,
        case when new_events.event_name = 'FathersDay' then 1 else 0 end as FathersDay,
        case when new_events.event_name = 'Halloween' then 1 else 0 end as Halloween,
        case when new_events.event_name = 'IndependenceDay' then 1 else 0 end as IndependenceDay,
        case when new_events.event_name = 'LaborDay' then 1 else 0 end as LaborDay,
        case when new_events.event_name = 'Lent' then 1 else 0 end as Lent,
        case when new_events.event_name = 'MartinLutherKingDay' then 1 else 0 end as MartinLutherKingDay,
        case when new_events.event_name = 'MemorialDay' then 1 else 0 end as MemorialDay,
        case when new_events.event_name = 'MothersDay' then 1 else 0 end as MothersDay,
        case when new_events.event_name = 'NBAFinal' then 1 else 0 end as NBAFinal,
        case when new_events.event_name = 'NewYear' then 1 else 0 end as NewYear,
        case when new_events.event_name = 'OrthodoxChristmas' then 1 else 0 end as OrthodoxChristmas,
        case when new_events.event_name = 'OrthodoxEaster' then 1 else 0 end as OrthodoxEaster,
        case when new_events.event_name = 'Pesach' then 1 else 0 end as Pesach,
        case when new_events.event_name = 'PresidentsDay' then 1 else 0 end as PresidentsDay,
        case when new_events.event_name = 'Purim' then 1 else 0 end as Purim,
        case when new_events.event_name = 'Ramadan' then 1 else 0 end as Ramadan,
        case when new_events.event_name = 'StPatricksDay' then 1 else 0 end as StPatricksDay,
        case when new_events.event_name = 'SuperBowl' then 1 else 0 end as SuperBowl,
        case when new_events.event_name = 'Thanksgiving' then 1 else 0 end as Thanksgiving,
        case when new_events.event_name = 'ValentinesDay' then 1 else 0 end as ValentinesDay,
        case when new_events.event_name = 'VeteransDay' then 1 else 0 end as VeteransDay
    from tmp_calendar  
    left outer join new_events
     on 
        tmp_calendar.date between new_events.from_day and new_events.to_day
    )
    group by date
    order by date
    '''
event_code_calendar = pd.read_sql_query(qry, conn)

In [90]:
event_code_calendar.head()

,date,E01,E02,E03,E04,E05,E06,E07,E08,E09,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E20,E21,E22,E23,E24,E25,E26,E27,E28
0,2011-01-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2011-01-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2011-01-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2011-02-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2011-02-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
event_code_calendar.to_csv('/content/event_code_calendar.csv')